In [0]:
dynamic view


databricks docs

data ingestion (theory) and step

schema==== BI-- Power BI

Databricks CLI 
web
vs code

DABs 

Machine learning

AI

In [0]:
%sql
use catalog auto;
use manju_silver;

In [0]:
%sql
CREATE OR REPLACE TABLE heartrate_device (device_id INT, mrn STRING, name STRING, time TIMESTAMP, heartrate DOUBLE);

INSERT INTO heartrate_device VALUES
  (23, "40580129", "Nicholas Spears", "2020-02-01T00:01:58.000+0000", 54.0122153343),
  (17, "52804177", "Lynn Russell", "2020-02-01T00:02:55.000+0000", 92.5136468131),
  (37, "65300842", "Samuel Hughes", "2020-02-01T00:08:58.000+0000", 52.1354807863),
  (23, "40580129", "Nicholas Spears", "2020-02-01T00:16:51.000+0000", 54.6477014191),
  (17, "52804177", "Lynn Russell", "2020-02-01T00:18:08.000+0000", 95.033344842);

In [0]:
%sql
SELECT * FROM heartrate_device;

In [0]:
%sql
CREATE OR REPLACE VIEW manju_gold.agg_heartrate AS (
  SELECT mrn, name, MEAN(heartrate) avg_heartrate, DATE_TRUNC("DD", time) date
  FROM heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);
SELECT * FROM naval_gold.agg_heartrate

In [0]:
%sql
CREATE OR REPLACE VIEW manju_gold.agg_heartrate AS (
  SELECT 
  CASE WHEN
    is_account_group_member('de') THEN mrn
    ELSE 'REDACTED'
  END AS mrn, 
  name, 
  MEAN(heartrate) avg_heartrate, 
  DATE_TRUNC("DD", time) date
  FROM heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);

In [0]:
%sql
SELECT * FROM manju_gold.agg_heartrate;

In [0]:
%sql
select * FROM auto.manju_silver.heartrate_device

In [0]:
%sql
CREATE OR REPLACE VIEW manju_gold.agg_heartrate AS (
  SELECT 
  CASE WHEN
    is_account_group_member('account users') THEN 'REDACTED'
    ELSE mrn
  END AS mrn, 
  name, 
  MEAN(heartrate) avg_heartrate, 
  DATE_TRUNC("DD", time) date
  FROM heartrate_device
  where
  CASE
    WHEN is_account_group_member('account users') THEN heartrate>53
    ELSE True
  END
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);

In [0]:
%sql
SELECT 
  CASE 
    WHEN is_account_group_member('account users') THEN 'REDACTED'
    ELSE mrn 
  END AS mrn,
  device_id
FROM auto.manju_silver.heartrate_device
WHERE 
  (
    is_account_group_member('account users') AND device_id > 17
  )
  OR
  (
    NOT is_account_group_member('account users')
  );

In [0]:
%sql
CREATE OR REPLACE FUNCTION datamask(x STRING)
  RETURNS STRING
  RETURN CONCAT(REPEAT("*", LENGTH(x) - 2), RIGHT(x, 2)
); 

In [0]:
%sql
select datamask("naval@gmail.com") as mask

In [0]:
%sql
CREATE OR REPLACE VIEW manju_gold.agg_heartrate AS (
  SELECT 
  CASE WHEN
    is_account_group_member('account users') THEN datamask(mrn)
    ELSE mrn
  END AS mrn, 
  name, 
  MEAN(heartrate) avg_heartrate, 
  DATE_TRUNC("DD", time) date
  FROM heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);

select * from manju_gold.agg_heartrate;